<a href="https://colab.research.google.com/github/chinhegde/Data-Wrangling-Practice/blob/main/Practice_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test 1

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'age': [25, 35, 45] * 10})

In [ ]:
df['age'].head()

,age
0,25
1,35
2,45
3,25
4,35


In [ ]:
df['age'] = (df['age'].head()*3) + 5

In [ ]:
df['age'].head()

,age
0,80.0
1,110.0
2,140.0
3,80.0
4,110.0


In [ ]:
# All combinations
# n= 3 -> 1 2 3, k = 2
# output = (1,2), (2,3), (1, 3)

from itertools import combinations

def func(n, k):
  return list(combinations(list(range(1, n+1)), k))

In [ ]:
func(3, 2)

[(1, 2), (1, 3), (2, 3)]

In [ ]:
func(4, 3)

[(1, 2, 3), (1, 2, 4), (1, 3, 4), (2, 3, 4)]

In [ ]:
def func1(n, k):

  res = list()

  def create_lists(i, k, curr):

    if k == 0:
      res.append(curr.copy())
      return

    if i > n:
      return

    curr.append(i)
    create_lists(i+1, k-1, curr)
    curr.pop()
    create_lists(i+1, k, curr)

  create_lists(1, k, curr = list())

  print("RES", res)

  return res

In [ ]:
func1(4, 3)

RES [[1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]


[[1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]

In [ ]:
# user session data - user_id, session_start, session_end, actions

def clean_session_data(data):

  data['session_start'] = pd.to_datetime(data['session_start'])
  data['session_end'] = pd.to_datetime(data['session_end'])

  data['is_valid_session'] = data['session_start'] < data['session_end']

  return data

In [ ]:
# viewership data - stream_id, user_id, viewers (count), timestamp
# max number of users per stream over time

def get_max_stream(df):

  res = (
      df.groupby(by='stream_id')['viewers']
      .max()
      .reset_index()
      .rename(columns={'viewers':'max_viewers_per_stream'})
    )

  return res

In [ ]:
data = {
    'stream_id': [1, 1, 1, 2, 2, 2],
    'user_id': [101, 102, 103, 201, 202, 203],
    'viewers': [150, 200, 180, 250, 300, 280],
    'timestamp': [
        '2024-08-10 10:00:00',
        '2024-08-10 10:05:00',
        '2024-08-10 10:10:00',
        '2024-08-10 11:00:00',
        '2024-08-10 11:05:00',
        '2024-08-10 11:10:00'
    ]
}

data = pd.DataFrame(data)

get_max_stream(data)

,stream_id,max_viewers_per_stream
0,1,200
1,2,300


In [ ]:
# User interactions data

data = {
    'user_id': [1, 1, 2, 2, 1, 2, 3, 3],
    'stream_id': [101, 101, 102, 102, 103, 103, 104, 104],
    'action_type': ['view', 'chat', 'view', 'like', 'like', 'chat', 'view', 'view'],
    'timestamp': [
        '2024-08-10 10:00:00',
        '2024-08-10 10:01:00',
        '2024-08-10 10:05:00',
        '2024-08-10 10:06:00',
        '2024-08-10 10:10:00',
        '2024-08-10 10:15:00',
        '2024-08-10 11:00:00',
        '2024-08-10 11:01:00'
    ]
}

data = pd.DataFrame(data)

In [ ]:
# create a feature matrix
# row -> user_id and count of each action

def feature_engineering(df):

    feature_matrix = pd.pivot_table(df, index='user_id', columns='action_type', aggfunc='size', fill_value=0)
    feature_matrix.reset_index(inplace=True)

    return feature_matrix

In [ ]:
feature_engineering(data)

action_type,user_id,chat,like,view
0,1,1,1,1
1,2,1,1,1
2,3,0,0,2


Churn Prediction Feature Engineering:
Question: Suppose you have a dataset of user interactions on Twitch (with columns user_id, stream_id, action_type, timestamp).
You need to build features for a churn prediction model. Write a Python function that creates the following features for each user:
- The total number of unique streams a user interacted with.
- The number of days since the user's last interaction.
- The ratio of views to total interactions per user.

In [ ]:
data = {
    'user_id': [1, 1, 2, 2, 1, 2, 3, 3],
    'stream_id': [101, 101, 102, 102, 103, 103, 104, 104],
    'action_type': ['view', 'chat', 'view', 'like', 'like', 'chat', 'view', 'view'],
    'timestamp': [
        '2024-08-10 10:00:00',
        '2024-08-10 10:01:00',
        '2024-08-10 10:05:00',
        '2024-08-10 10:06:00',
        '2024-08-10 10:10:00',
        '2024-08-10 10:15:00',
        '2024-08-10 11:00:00',
        '2024-08-10 11:01:00'
    ]
}

df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
from datetime import datetime

def churn_features(df):

  # streams per user -> count(stream_id)
  # last interaction = datetime.now() - timestamp
  # ratio of views = view count/total actions

  unique_streams = (
      df.groupby('user_id')['stream_id']
      .nunique()
      .reset_index()
      .rename(columns={'stream_id':'unique_streams'})
      )

  total_streams = (
      df.groupby('user_id')['stream_id']
      .count()
      .reset_index()
      .rename(columns={'stream_id':'total_streams'})
      )

  last_interaction = df.groupby('user_id')['timestamp'].max().reset_index()
  last_interaction['days_since_last'] = (datetime.now() - last_interaction['timestamp']).dt.days

  last_interaction.drop(['timestamp'], axis = 1, inplace=True)

  views_per_user = (
      df[df['action_type'] == 'view']
      .groupby('user_id')['stream_id']
      .count()
      .reset_index()
      .rename(columns={'stream_id':'views_count'})
      )

  res = pd.merge(unique_streams, last_interaction, on='user_id')
  res = pd.merge(res, total_streams, on='user_id')
  res = pd.merge(views_per_user, res, on='user_id')

  res['view_to_total_ratio'] = round(res['views_count']/res['total_streams'], 2)
  res.drop(['views_count', 'total_streams'], axis = 1, inplace=True)

  return res

In [ ]:
churn_features(df)

,user_id,unique_streams,days_since_last,view_to_total_ratio
0,1,2,3,0.33
1,2,2,3,0.33
2,3,1,3,1.00


# Test 2

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Function to generate dummy data
def generate_dummy_data():
    np.random.seed(42)  # For reproducibility

    # Define parameters
    num_users = 3
    num_days = 5
    max_interactions_per_day = 20
    start_date = datetime(2024, 8, 10)

    data = {
        'user_id': [],
        'stream_id': [],
        'timestamp': [],
        'action_type': []
    }

    for user_id in range(1, num_users + 1):
        for day in range(num_days):
            # Generate a random number of interactions for the day
            interactions_today = np.random.randint(5, max_interactions_per_day + 1)
            date = start_date + timedelta(days=day)

            for interaction in range(interactions_today):
                # Random timestamp within the day
                random_seconds = np.random.randint(0, 86400)  # seconds in a day
                timestamp = date + timedelta(seconds=random_seconds)

                # Random action type
                action_type = np.random.choice(['view', 'like', 'share'])

                # Append to data
                data['user_id'].append(user_id)
                data['stream_id'].append(np.random.randint(100, 200))  # random stream_id
                data['timestamp'].append(timestamp)
                data['action_type'].append(action_type)

    return pd.DataFrame(data)

# Generate the data
df = generate_dummy_data()

In [ ]:
df.head()

,user_id,stream_id,timestamp,action_type,day
0,1,114,2024-08-10 04:23:15,view,2024-08-10
1,1,182,2024-08-10 21:20:20,share,2024-08-10
2,1,187,2024-08-10 10:19:54,share,2024-08-10
3,1,121,2024-08-10 12:15:31,share,2024-08-10
4,1,187,2024-08-10 18:00:20,like,2024-08-10


9. Identifying Content Consumption Patterns:
Question: Given a dataset with columns user_id, stream_id, timestamp, and action_type (where action_type can be view, like, share), write a Python function to identify users who are "power users."

A power user is defined as someone who has engaged with the content (either by viewing, liking, or sharing) more than 10 times in a single day on at least 3 different days. The function should return a list of user_ids who meet this criterion.

In [ ]:
def identify_power_users(df):

  df['day'] = df['timestamp'].dt.date

  users_agg = (
      df.groupby(['user_id', 'day'])['action_type']
      .count()
      .reset_index()
      .rename(columns={'action_type':'action_count'})
      )

  power_users1 = users_agg[users_agg['action_count'] > 10]

  power_users2 = (
      power_users1.groupby('user_id')['day']
      .count()
      .reset_index()
  )
  power_users2 = power_users2[power_users2['day'] >= 3]['user_id'].unique()

  power_users = set(power_users1['user_id'].unique()).intersection(set(power_users2))

  return pd.DataFrame({'user_id':list(power_users)})

In [ ]:
res_df = identify_power_users(df)
print(res_df)

   user_id
0        1
1        2


In [ ]:
def validate_power_users(df, res_df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date

    interaction_counts = df.groupby(['user_id', 'date']).size().reset_index(name='interaction_count')

    frequent_users = interaction_counts[interaction_counts['interaction_count'] > 10]

    active_days_per_user = frequent_users.groupby('user_id').size().reset_index(name='active_days')

    expected_users = active_days_per_user[active_days_per_user['active_days'] >= 3]['user_id'].tolist()

    valid = set(res_df['user_id']) == set(expected_users)

    if valid:
        print("Validation successful: The result is correct.")
    else:
        print("Validation failed: There are discrepancies.")
        print("Expected Users:", expected_users)
        print("Result Users:", res_df['user_id'].tolist())

# Assuming res_df is your result DataFrame
validate_power_users(df, res_df)

Validation successful: The result is correct.


8. Streaming Frequency Analysis:
Question: You have a dataset with columns stream_id, user_id, timestamp, and action_type (where action_type can be start or end).

Write a Python function that calculates the **average stream duration** and the **frequency of streaming** for each user. Assume that every start action is followed by an end action, and that timestamps are ordered.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

def generate_dummy_data(num_users=100, num_streams_per_user=10):
    data = []
    start_time = datetime(2024, 8, 10, 10, 0, 0)

    for user_id in range(1, num_users + 1):
        for stream_id in range(num_streams_per_user):
            stream_start = start_time + timedelta(minutes=random.randint(1, 1000))
            stream_end = stream_start + timedelta(minutes=random.randint(5, 120))

            data.append({
                'stream_id': stream_id + user_id * 1000,
                'user_id': user_id,
                'timestamp': stream_start.strftime('%Y-%m-%d %H:%M:%S'),
                'action_type': 'start'
            })

            data.append({
                'stream_id': stream_id + user_id * 1000,
                'user_id': user_id,
                'timestamp': stream_end.strftime('%Y-%m-%d %H:%M:%S'),
                'action_type': 'end'
            })

    df = pd.DataFrame(data)
    df = df.sort_values(by=['user_id', 'timestamp'])
    return df

# Generate dummy data
df = generate_dummy_data()

In [ ]:
df.head()

,stream_id,user_id,timestamp,action_type
14,1007,1,2024-08-10 10:42:00,start
15,1007,1,2024-08-10 11:30:00,end
12,1006,1,2024-08-10 11:38:00,start
18,1009,1,2024-08-10 11:50:00,start
8,1004,1,2024-08-10 12:13:00,start


In [ ]:
def stream_analysis(df):

  temp = df[['user_id', 'stream_id']].drop_duplicates()
  temp.sort_values(by='user_id', inplace=True)

  temp['start_time'] = list(df[df['action_type']=='start'].sort_values(by='user_id')['timestamp'])
  temp['end_time'] = list(df[df['action_type']=='end'].sort_values(by='user_id')['timestamp'])

  temp['end_time'] = pd.to_datetime(temp['end_time'])
  temp['start_time'] = pd.to_datetime(temp['start_time'])

  temp['duration'] = (temp['end_time'] - temp['start_time'])
  temp['duration'] = temp['duration'].dt.total_seconds() / 60

  return (
      temp.groupby(by='user_id')
      .agg({'duration': 'mean',
            'stream_id': 'size'})
      .reset_index()
      .rename(columns={'duration':'avg_duration', 'stream_id': 'stream_frequency'})
  )

In [ ]:
res = stream_analysis(df)
res

,user_id,avg_duration,stream_frequency
0,1,60.7,10
1,2,74.9,10
2,3,61.3,10
3,4,57.6,10
4,5,79.3,10
...,...,...,...
95,96,50.9,10
96,97,56.4,10
97,98,67.3,10
98,99,61.0,10


In [ ]:
import pandas as pd

data = {'user_id': [1, 1, 1, 1, 1, 1, 2, 2, 2],
        'timestamp': pd.to_datetime(['2024-08-01 12:00:00', '2024-08-01 12:01:00', '2024-08-01 12:02:00',
                                     '2024-08-01 12:03:00', '2024-08-01 12:04:00', '2024-08-01 12:05:00',
                                     '2024-08-01 12:00:00', '2024-08-01 12:01:00', '2024-08-01 12:02:00']),
        'message': ['msg1', 'msg2', 'msg3', 'msg4', 'msg5', 'msg6', 'msg1', 'msg2', 'msg3']}
df = pd.DataFrame(data)


In [ ]:
df.head()

,user_id,timestamp,message
0,1,2024-08-01 12:00:00,msg1
1,1,2024-08-01 12:01:00,msg2
2,1,2024-08-01 12:02:00,msg3
3,1,2024-08-01 12:03:00,msg4
4,1,2024-08-01 12:04:00,msg5


In [ ]:
df['count'] = df.groupby('user_id').cumcount() + 1
df.head()

,user_id,timestamp,message,count
0,1,2024-08-01 12:00:00,msg1,1
1,1,2024-08-01 12:01:00,msg2,2
2,1,2024-08-01 12:02:00,msg3,3
3,1,2024-08-01 12:03:00,msg4,4
4,1,2024-08-01 12:04:00,msg5,5


In [ ]:
df.set_index('timestamp').groupby('user_id')['count'].rolling('10T').count()

user_id  timestamp          
1        2024-08-01 12:00:00    1.0
         2024-08-01 12:01:00    2.0
         2024-08-01 12:02:00    3.0
         2024-08-01 12:03:00    4.0
         2024-08-01 12:04:00    5.0
         2024-08-01 12:05:00    6.0
2        2024-08-01 12:00:00    1.0
         2024-08-01 12:01:00    2.0
         2024-08-01 12:02:00    3.0
Name: count, dtype: float64